# Run a basic Hermes Hartree-Fock energy calculation on ligand conformers from Auto3d

In this notebook, we'll perform a hermes energy calculation on a ligand derived from a smiles string

# 0) Complete example
See the [sample notebook](/Quickstarts/auto3d_conformer_tautomer_generation-sample.ipynb) for a complete demonstration.

# 1) Setup

## 1.0) Imports

In [ ]:
import json
import os
import sys
import tarfile

from pdbtools import *
import requests
from datetime import datetime
from pathlib import Path
import py3Dmol

import rush

## 1.1) Configuration

In [ ]:
EXPERIMENT = "tengu-py-auto3d_hermes"
LIGAND_SMILES = "CC(=O)OC1=CC=CC=C1C(=O)O "
LIGAND = "ASPRIN"
TAGS = ["qdx", EXPERIMENT, LIGAND]

In [ ]:
# |hide
WORK_DIR = Path.home() / "qdx" / EXPERIMENT

if WORK_DIR.exists():
    client = rush.Provider(workspace=WORK_DIR)
    await client.nuke(remote=False)

os.makedirs(WORK_DIR, exist_ok=True)
os.makedirs(WORK_DIR / ".rush", exist_ok=True)

os.chdir(WORK_DIR)
YOUR_TOKEN = os.getenv("RUSH_TOKEN")

2024-03-21 09:51:56,554 - rush - INFO - Restoring by default via env
2024-03-21 09:52:13,909 - rush - WARNING - Module hermes_energy_batch has a different version on the server: github:talo/tengu-prelude/4a16cf12be35f0a40c113d4410046de865f1906f#hermes_energy_batch.
                                Use `.update_modules()` to update the lock file
2024-03-21 09:52:13,909 - rush - WARNING - Module hermes_energy has a different version on the server: github:talo/tengu-prelude/4a16cf12be35f0a40c113d4410046de865f1906f#hermes_energy.
                                Use `.update_modules()` to update the lock file
2024-03-21 09:52:13,910 - rush - WARNING - Module gmx has a different version on the server: github:talo/tengu-gmx/04cff2931b995c33263dfdb477d7f09c8bbd75a7#gmx_tengu.
                                Use `.update_modules()` to update the lock file
2024-03-21 09:52:13,910 - rush - WARNING - Module gmx_resume has a different version on the server: github:talo/tengu-gmx/04cff2931b995c33263d

## 1.2) Build your client

In [ ]:
# Get our client, for calling modules and using the rush API
client = rush.build_blocking_provider_with_functions(batch_tags=TAGS)

2024-03-21 09:51:56,579 - rush - INFO - Restoring by default via env


# 2) Preparation
We want to convert our raw files into prepared qdxfs, with correct charges and missing residues filled

In [ ]:
help(client.auto3d)

Help on function auto3d in module rush.provider:

auto3d(*args: *tuple[RushObject[bytes], str, Record], **kwargs) -> tuple[RushObject[None], RushObject[list[Record]]]
    Generate 3D conformers from SMILES strings and other inputs

    Module version:
    `github:talo/tengu-auto3d/bd31770c27581753010b6623cdd4bd82b0628e79#auto3d_tengu`

    QDX Type Description:

        molecule_file: Object[@$Bytes];
        molecule_file_type: string;
        options: Auto3dOptions {
            capacity: u32?,
            mpi_np: u32?,
            optimizing_engine: Auto3dOptimizingEngines[ANI2x | ANI2xt | AIMNET]?,
            opt_steps: u32?,
            patience: u32?,
            memory: u32?,
            enumerate_isomer: bool?,
            gpu_idx: [u32]?,
            job_name: string?,
            use_gpu: bool?,
            window: f32?,
            enumerate_tautomer: bool?,
            threshold: f32?,
            max_confs: u32?,
            verbose: bool?,
            convergence_thresho

In [ ]:
ligand_path = client.workspace / "aspirin.smiles"
with open(ligand_path, "w") as ligand_file:
    print(f"{LIGAND_SMILES} {LIGAND_SMILES}", file=ligand_file)

(conformer_sdf, conformer_qdxf) = client.auto3d(
    ligand_path,
    "smi",
    {"k": 5, "use_gpu": True},
    resources={"gpus": 1, "storage": "5", "storage_units": "GB"},
)

print(f"{datetime.now().time()} | Running ligand preparation!")

2024-03-21 09:51:57,869 - rush - INFO - Trying to restore job with tags: ['qdx', 'tengu-py-auto3d_hermes', 'ASPRIN'] and path: github:talo/tengu-auto3d/bd31770c27581753010b6623cdd4bd82b0628e79#auto3d_tengu
2024-03-21 09:51:57,919 - rush - INFO - Restoring job from previous run with id e6b68dfe-1908-41ad-8d6d-669455a68dcc
09:51:57.920521 | Running ligand preparation!


In [ ]:
conformer_qdxf.source

UUID('e6b68dfe-1908-41ad-8d6d-669455a68dcc')

In [ ]:
try:
    conformer_sdf.download(filename="01_prepared_ligand.sdf")
except FileExistsError:
    # we will raise an error if you try to overwrite an existing file; you can
    # force the file to overwrite by passing an absolute filepath instead
    pass

You should visualize your prepared ligand to spot check any issues

In [ ]:
view = py3Dmol.view()
with open(client.workspace / "objects" / "01_prepared_ligand.sdf", "r") as f:
    view.addModel(f.read(), "sdf")
    view.setStyle({"stick": {}})
    view.zoomTo()
    view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

# 3) Quantum energy calculation
Finally, we submit our fragmented protein for quantum energy calculation, with custom configuration.

In [ ]:
help(client.hermes_energy)

Help on function hermes_energy in module rush.provider:

hermes_energy(*args: *tuple[RushObject[Record], Record, Optional[Record], Optional[Record], Optional[Record], Optional[Record]], **kwargs) -> tuple[RushObject[Record]]
    Runs a HERMES energy calculation given a topology, and optionally model and keyword configurations.
    Will use the default model and keywords if none are provided

    Module version:
    `github:talo/tengu-prelude/4a16cf12be35f0a40c113d4410046de865f1906f#hermes_energy`

    QDX Type Description:

        input: Object[Conformer];
        system: System {
            max_gpu_memory_mb: u32?,
            teams_per_node: u32?,
            gpus_per_team: u32?,
            oversubscribe_gpus: bool?
        };
        model: Model {
            aux_basis: string?,
            standard_orientation: string?,
            basis: string,
            force_cartesian_basis_sets: bool?
        }?;
        scf keywords: SCF {
            convergence_metric: string?,
      

In [ ]:
HERMES_RESOURCES = {
    "gpus": 1,
    "storage": 100,
    "storage_units": "MB",
    "walltime": 60,
}

In [ ]:
(conformer,) = client.pick_conformer(conformer_qdxf, 0)

2024-03-21 09:52:11,037 - rush - INFO - Trying to restore job with tags: ['qdx', 'tengu-py-auto3d_hermes', 'ASPRIN'] and path: github:talo/tengu-prelude/b345a0b0077225c63d904d0e03fb9ca1acec55ed#pick_conformer
2024-03-21 09:52:11,083 - rush - INFO - Restoring job from previous run with id 5685e4a8-9375-4f6c-8714-4f45d7a4fecf


In [ ]:
conformer_out = json.load(open(conformer.download(overwrite=True), "r"))

In [ ]:
(hermes_energy,) = client.hermes_energy(
    conformer,
    {},
    {
        "method": "RestrictedRIMP2",
        "basis": "cc-pVDZ",
        "aux_basis": "cc-pVDZ-RIFIT",
        "force_cartesian_basis_sets": True,
        "standard_orientation": "FullSystem",
    },
    None,
    None,
    None,
    resources=HERMES_RESOURCES,
)

In [ ]:
json.load(hermes_energy.download().open())

2024-03-21 09:52:47,002 - rush - INFO - Argument 5adf6fc7-6fac-4cc5-8f5f-84fa509862c0 is now ModuleInstanceStatus.RESOLVING
2024-03-21 09:52:50,393 - rush - INFO - Argument 5adf6fc7-6fac-4cc5-8f5f-84fa509862c0 is now ModuleInstanceStatus.ADMITTED
2024-03-21 09:53:45,191 - rush - INFO - Argument 5adf6fc7-6fac-4cc5-8f5f-84fa509862c0 is now ModuleInstanceStatus.DISPATCHED
2024-03-21 09:53:51,809 - rush - INFO - Argument 5adf6fc7-6fac-4cc5-8f5f-84fa509862c0 is now ModuleInstanceStatus.RUNNING
2024-03-21 09:54:01,977 - rush - INFO - Argument 5adf6fc7-6fac-4cc5-8f5f-84fa509862c0 is now ModuleInstanceStatus.AWAITING_UPLOAD


{'distance_metric': None,
 'expanded_density': None,
 'expanded_hf_energy': -645.0201870898042,
 'expanded_hf_gradients': None,
 'expanded_mp2_gradients': None,
 'expanded_mp2_os_correction': -1.4322053052020325,
 'expanded_mp2_ss_correction': -0.5339028327148518,
 'method': 'RestrictedRIMP2',
 'nmers': [[{'bond_orders': None,
    'coeffs_final': None,
    'coeffs_initial': None,
    'delta_hf_energy': None,
    'delta_mp2_os_correction': None,
    'delta_mp2_ss_correction': None,
    'density': None,
    'fock': None,
    'fragment_distance': None,
    'fragments': [0],
    'h_caps': None,
    'h_core': None,
    'hf_energy': -645.0201870898042,
    'hf_gradients': None,
    'molecular_orbital_energies': None,
    'mp2_gradients': None,
    'mp2_os_correction': -1.4322053052020325,
    'mp2_ss_correction': -0.5339028327148518,
    'mulliken_charges': None,
    'num_basis_fns': 235,
    'num_iters': 9,
    'overlap': None}]],
 'num_iters': 9,
 'reference_fragment': None}